In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("diamonds.csv")

In [4]:
df.shape

(53940, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [6]:
df = df.drop('Unnamed: 0',axis=1)

In [7]:
df_ohe = pd.get_dummies(df)

In [8]:
df_ohe.shape

(53940, 27)

# Outlier Handling

## Z-score based / Standard Scaler based method

In [9]:
num_columns = df.select_dtypes(exclude='object').columns

In [10]:
num_columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z'], dtype='object')

In [11]:
def zscore_for_column(df,col):
    #zscore = (val - mean) / std_dev
    zscore = (df[col] - df[col].mean())/df[col].std()
    return zscore

In [12]:
df_std = df_ohe.copy()
for col in num_columns:
    df_std[col] = zscore_for_column(df,col)

In [13]:
df_std.describe()

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,5.394000e+04,53940.000000,53940.000000,53940.000000,...,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,2.571337e-16,-4.002039e-15,1.264592e-16,-5.901430e-17,2.782103e-16,-1.159210e-16,-1.686123e-16,0.029848,0.090953,0.399537,...,0.100519,0.052058,0.013737,0.033185,0.242214,0.170449,0.151483,0.227253,0.067760,0.093919
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.170169,0.287545,0.489808,...,0.300694,0.222146,0.116400,0.179121,0.428427,0.376030,0.358522,0.419061,0.251337,0.291719
min,-1.261446e+00,-1.308748e+01,-6.470013e+00,-9.040868e-01,-5.109073e+00,-5.020884e+00,-5.014510e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.395154e-01,-5.231005e-01,-6.521325e-01,-7.476738e-01,-9.103164e-01,-8.882717e-01,-8.909378e-01,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-2.066190e-01,3.531645e-02,-2.046032e-01,-3.839636e-01,-2.777527e-02,-2.147379e-02,-1.237607e-02,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.106635e-01,5.239313e-01,6.904554e-01,3.487834e-01,7.210475e-01,7.052356e-01,7.103118e-01,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.885992e+00,1.204128e+01,1.680151e+01,3.732404e+00,4.465161e+00,4.654922e+01,4.004720e+01,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
df_std.loc[(df_std['carat'] < -3) | (df_std['carat'] > +3), :].shape

(439, 27)

In [17]:
from IPython.utils.text import columnize
def outlier_imputation(df,col):
    df.loc[df[col] < -3 ,col] = -3 
    df.loc[df[col] > 3 ,col] = 3
    return df

In [20]:
for col in num_columns:
    df_std = outlier_imputation(df_std,col)

In [21]:
df_std.describe()

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,...,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.016748,0.095729,0.055750,0.010303,0.030939,0.032093,0.033689,1.700408,1.767204,2.074898,...,1.776103,1.731535,1.693845,1.715795,1.916073,1.839878,1.829347,1.902595,1.750019,1.775695
std,1.080181,1.064749,1.111425,1.053122,1.115840,1.097247,1.101705,19.874702,19.870555,19.848537,...,19.869985,19.872797,19.875097,19.873766,19.860500,19.865787,19.866494,19.861457,19.871643,19.870011
min,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.839515,-0.453298,-0.652132,-0.749930,-0.919231,-0.897027,-0.905108,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-0.206619,0.105119,-0.204603,-0.382710,-0.036690,-0.021474,-0.019461,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.531760,0.593733,0.690455,0.382309,0.738877,0.713991,0.710312,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,253.000000,253.000000,253.000000,...,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000


## IQR Based outlier handling

In [15]:
df_iqr = df_ohe.copy()

In [16]:
# imputation formula
# IQR = Q3 - Q1
# min_val_value = Q1 - 1.5 * IQR
# max_val_value = Q3 + 1.5 * IQR
# val < min_val_value  --> min_val_value
# val > max_val_value  --> max_val_value

In [17]:
q1, q3 = df['table'].quantile([0.25,0.75])

In [18]:
q1, q3

(56.0, 59.0)

In [19]:
def outlier_imputation_IQR(df,col):
    q1, q3 = df[col].quantile([0.25,0.75])
    iqr = q3 -q1

    df.loc[df[col] < (q1-1.5*iqr),col ] = (q1-1.5*iqr) 
    df.loc[df[col] > (q3+1.5*iqr),col ] = (q3+1.5*iqr)
    return df

In [20]:
for col in num_columns:
    df_iqr = outlier_imputation_IQR(df_iqr,col)

In [21]:
df_iqr.describe()

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,...,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.792558,61.749156,57.438561,3732.145690,5.731285,5.733311,3.538500,0.029848,0.090953,0.399537,...,0.100519,0.052058,0.013737,0.033185,0.242214,0.170449,0.151483,0.227253,0.067760,0.093919
std,0.457089,1.270254,2.161139,3436.769344,1.119956,1.111950,0.692494,0.170169,0.287545,0.489808,...,0.300694,0.222146,0.116400,0.179121,0.428427,0.376030,0.358522,0.419061,0.251337,0.291719
min,0.200000,58.750000,51.500000,326.000000,1.965000,1.990000,1.215000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,64.750000,63.500000,11885.625000,9.285000,9.270000,5.735000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
dff = pd.read_csv("Sonar.csv")

In [40]:
dff.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [41]:
#dff_sonar = pd.get_dummies(dff)

In [42]:
dff_sonar.shape

(208, 62)

In [43]:
num_columns = dff.select_dtypes(exclude='object').columns

In [44]:
num_columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31',
       'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41',
       'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51',
       'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60'],
      dtype='object')

In [45]:
def zscore_for_column(dff,col):
    #zscore = (val - mean) / std_dev
    zscore = (dff[col] - dff[col].mean())/dff[col].std()
    return zscore

In [49]:
dff_std = dff_sonar.copy()
for col in num_columns:
    dff_std[col] = zscore_for_column(dff,col)

In [50]:
dff_std.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V53,V54,V55,V56,V57,V58,V59,V60,Class_M,Class_R
count,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,...,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,208.000000,208.000000
mean,-1.708035e-17,6.832142e-17,-8.967186e-17,1.708035e-16,-1.708035e-16,2.476651e-16,7.686159e-17,3.416071e-17,-3.757678e-16,3.416071e-17,...,5.124106e-17,-1.281027e-16,4.270089e-18,-2.391250e-16,3.843080e-17,-1.281027e-16,1.409129e-16,6.405133e-17,0.533654,0.466346
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500070,0.500070
min,-1.203255e+00,-1.147956e+00,-1.101595e+00,-1.033622e+00,-1.233118e+00,-1.596641e+00,-1.916988e+00,-1.518447e+00,-1.440214e+00,-1.465298e+00,...,-1.445984e+00,-1.361612e+00,-1.226134e+00,-1.363578e+00,-1.299835e+00,-1.182261e+00,-1.268542e+00,-1.174152e+00,0.000000,0.000000
25%,-6.878344e-01,-6.670687e-01,-6.475002e-01,-6.343993e-01,-6.687841e-01,-6.352240e-01,-6.610783e-01,-6.385513e-01,-6.840088e-01,-7.215237e-01,...,-7.979979e-01,-7.623633e-01,-7.252614e-01,-6.662415e-01,-7.121589e-01,-6.722018e-01,-6.901929e-01,-6.772376e-01,0.000000,0.000000
50%,-2.768025e-01,-2.316916e-01,-2.480531e-01,-2.115353e-01,-2.286572e-01,-2.101362e-01,-2.394747e-01,-2.665702e-01,-2.175310e-01,-1.923818e-01,...,-1.641755e-01,-2.247513e-01,-2.526070e-01,-2.391228e-01,-3.232553e-01,-3.321625e-01,-2.493530e-01,-2.399525e-01,1.000000,0.000000
75%,2.777644e-01,2.886372e-01,3.673818e-01,2.279853e-01,4.513343e-01,5.000353e-01,5.220014e-01,4.086914e-01,4.681428e-01,4.496562e-01,...,5.935786e-01,4.874989e-01,3.964111e-01,4.102720e-01,4.502307e-01,3.711006e-01,3.856182e-01,4.010676e-01,1.000000,1.000000
max,4.694727e+00,5.930336e+00,6.819689e+00,8.006103e+00,5.864714e+00,4.698888e+00,4.064767e+00,3.807312e+00,4.263950e+00,3.737218e+00,...,4.007013e+00,3.322803e+00,4.995974e+00,5.435454e+00,4.784345e+00,5.572156e+00,4.603930e+00,7.432412e+00,1.000000,1.000000


In [52]:
dff_iqr = dff_sonar.copy()

In [56]:
q1, q3 = dff[].quantile([0.25,0.75])

KeyError: 'Class_M'

In [54]:
q1, q3

(56.0, 59.0)